In [1]:
!pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\pintu\OneDrive\Desktop\Projects\car_price_predictor\outliers_treatment.csv')

In [3]:
df.sample()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
1496,1505,Volkswagen Polo 1.2 MPI Highline,Coimbatore,2014,72465,Petrol,Manual,First,23.058,1198.0,74.0,5.0,5.9


In [4]:
df['Name']

0       Hyundai Creta 1.6 CRDi SX Option
1                           Honda Jazz V
2                      Maruti Ertiga VDI
3        Audi A4 New 2.0 TDI Multitronic
4                 Nissan Micra Diesel XV
                      ...               
5801                    Maruti Swift VDI
5802            Hyundai Xcent 1.1 CRDi S
5803               Mahindra Xylo D4 BSIV
5804                  Maruti Wagon R VXI
5805               Chevrolet Beat Diesel
Name: Name, Length: 5806, dtype: object

In [18]:
!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/95/b3/10e875c45c60b010b66fc109b899c6fc4f05d485fe1d54abff98ce791124/regex-2024.7.24-cp312-cp312-win_amd64.whl.metadata
  Using cached regex-2024.7.24-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/18/eb/fdb7eb9e48b7b02554e1664afd3bd3f117f6b6d6c5881438a0b055554f9b/tqdm-4.66.4-py3-none-any.whl.metadata
  

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Download necessary resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize the PorterStemmer and stopwords
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def transform_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text
    text = nltk.word_tokenize(text)
    
    y = []
    
    # Remove non-alphanumeric characters
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    # Remove stopwords and punctuation
    for i in text:
        if i not in stop_words and i not in string.punctuation:
            y.append(i)
            
    text = y[:]
    y.clear()
    
    # Apply stemming
    for word in text:
        y.append(ps.stem(word))
    
    # Join the list of words back into a single string
    return " ".join(y)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pintu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pintu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df['transformed_name'] = df['Name'].apply(transform_text)

In [7]:
df['transformed_name']

0       hyundai creta crdi sx option
1                       honda jazz v
2                  maruti ertiga vdi
3          audi a4 new tdi multitron
4             nissan micra diesel xv
                    ...             
5801                maruti swift vdi
5802              hyundai xcent crdi
5803           mahindra xylo d4 bsiv
5804              maruti wagon r vxi
5805           chevrolet beat diesel
Name: transformed_name, Length: 5806, dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [9]:
X = tfidf.fit_transform(df['transformed_name'])

In [10]:
tfidf_df = pd.DataFrame(X.toarray(), columns=tfidf.get_feature_names_out())

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
cosine_sim1 = cosine_similarity(X,X)

In [13]:
cosine_sim1.shape

(5806, 5806)

In [14]:
def recommend_cars_with_model_name(model_name, cosine_sim=cosine_sim1):
    # Get the index of the car that matches the name
    idx = df.index[df['Name'] == model_name].tolist()[0]

    # Get the pairwise similarity scores with that car
    sim_scores = list(enumerate(cosine_sim1[idx]))

    # Sort the pcars based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar cars
    sim_scores = sim_scores[1:6]

    # Get the car indices
    car_indices = [i[0] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['Name'].iloc[car_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar cars
    return recommendations_df

In [15]:
recommend_cars_with_model_name('Maruti Wagon R VXI')

,PropertyName,SimilarityScore
477,Maruti Wagon R VXI,"(477, 1.0)"
760,Maruti Wagon R VXI,"(760, 1.0)"
1019,Maruti Wagon R VXI,"(1019, 1.0)"
1058,Maruti Wagon R VXI,"(1058, 1.0)"
1065,Maruti Wagon R VXI,"(1065, 1.0)"


In [16]:
df.sample()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,transformed_name
2988,3011,Hyundai i20 Magna Optional 1.4 CRDi,Ahmedabad,2012,55362,Diesel,Manual,First,30.66,1396.0,88.8,5.0,4.25,hyundai i20 magna option crdi


In [17]:
from sklearn.preprocessing import  OrdinalEncoder,StandardScaler

In [18]:
encoder = OrdinalEncoder()

In [19]:
df['Fuel_Type'] = encoder.fit_transform(df[['Fuel_Type']])
df['Owner_Type'] = encoder.fit_transform(df[['Owner_Type']])
df['Transmission'] = encoder.fit_transform(df[['Transmission']])

In [20]:
scaler = StandardScaler()

In [21]:
df.drop(columns=['Name','Location'],axis=1)

,Unnamed: 0,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,transformed_name
0,0,2015,41000,0.0,1.0,0.0,27.538,1582.0,126.20,5.0,12.50,hyundai creta crdi sx option
1,1,2011,46000,1.0,1.0,0.0,18.200,1199.0,88.70,5.0,4.50,honda jazz v
2,2,2012,87000,0.0,1.0,0.0,29.078,1248.0,88.76,7.0,6.00,maruti ertiga vdi
3,3,2013,40670,0.0,0.0,2.0,21.280,1968.0,140.80,5.0,17.74,audi a4 new tdi multitron
4,4,2013,86999,0.0,1.0,0.0,32.312,1461.0,63.10,5.0,3.50,nissan micra diesel xv
...,...,...,...,...,...,...,...,...,...,...,...,...
5801,5842,2014,27365,0.0,1.0,0.0,39.760,1248.0,74.00,5.0,4.75,maruti swift vdi
5802,5843,2015,100000,0.0,1.0,0.0,34.160,1120.0,71.00,5.0,4.00,hyundai xcent crdi
5803,5844,2012,55000,0.0,1.0,2.0,19.600,2498.0,112.00,8.0,2.90,mahindra xylo d4 bsiv
5804,5845,2013,46000,1.0,1.0,0.0,26.460,998.0,67.10,5.0,2.65,maruti wagon r vxi


In [22]:
df_normalized = pd.DataFrame(scaler.fit_transform(df.drop(columns=['Name','Location','transformed_name'],axis=1)), columns=df.drop(columns=['Name','Location','transformed_name'],axis=1).columns)

In [23]:
df_normalized

,Unnamed: 0,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,-1.730431,0.480952,-0.428407,-0.917927,0.643718,-0.455818,0.342059,-0.082461,0.230131,-0.355239,0.251681
1,-1.729839,-0.780726,-0.296504,1.089411,0.643718,-0.455818,-1.269278,-0.719141,-0.465986,-0.355239,-0.458254
2,-1.729247,-0.465306,0.785095,-0.917927,0.643718,-0.455818,0.607796,-0.637686,-0.464872,2.123749,-0.325141
3,-1.728655,-0.149887,-0.437112,-0.917927,-1.553476,2.022161,-0.737803,0.559207,0.501152,-0.355239,0.716688
4,-1.728063,-0.149887,0.785068,-0.917927,0.643718,-0.455818,1.165846,-0.283605,-0.941202,-0.355239,-0.546996
...,...,...,...,...,...,...,...,...,...,...,...
5801,1.727339,0.165533,-0.788104,-0.917927,0.643718,-0.455818,2.451050,-0.637686,-0.738864,-0.355239,-0.436069
5802,1.727930,0.480952,1.128041,-0.917927,0.643718,-0.455818,1.484731,-0.850467,-0.794553,-0.355239,-0.502625
5803,1.728522,-0.465306,-0.059080,-0.917927,0.643718,2.022161,-1.027698,1.440253,-0.033466,3.363243,-0.600241
5804,1.729114,-0.149887,-0.296504,1.089411,0.643718,-0.455818,0.156042,-1.053274,-0.866949,-0.355239,-0.622426


In [24]:
cosine_sim2 = cosine_similarity(df_normalized)

In [25]:
def recommend_cars_with_featurs(model_name, cosine_sim=cosine_sim2):
    # Get the index of the car that matches the name
    idx = df.index[df['Name'] == model_name].tolist()[0]

    # Get the pairwise similarity scores with that car
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the pcars based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar cars
    sim_scores = sim_scores[1:6]

    # Get the car indices
    car_indices = [i[0] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['Name'].iloc[car_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar cars
    return recommendations_df

In [26]:
recommend_cars_with_featurs('Hyundai i10 Magna 1.2')

,PropertyName,SimilarityScore
283,Hyundai i10 Magna 1.2 iTech SE,"(283, 0.9795021004802975)"
673,Maruti Ritz LXi,"(673, 0.978316631241261)"
1207,Hyundai i10 Sportz,"(1207, 0.9751208354978083)"
548,Maruti Swift ZXI,"(548, 0.9739258220660814)"
1046,Hyundai i10 Magna 1.2,"(1046, 0.9719093924203148)"


In [27]:
df.iloc[[181,187,207,281,341]]

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,transformed_name
181,181,Hyundai i10 Magna 1.1,Kochi,2017,52340,1.0,1.0,0.0,27.734,1086.0,68.05,5.0,4.04,hyundai i10 magna
187,188,Hyundai i10 Magna,Jaipur,2015,43571,1.0,1.0,0.0,28.504,1197.0,78.90,5.0,3.55,hyundai i10 magna
207,209,Hyundai i10 Magna 1.1,Pune,2012,44000,1.0,1.0,2.0,27.734,1086.0,68.05,5.0,2.90,hyundai i10 magna
281,284,Hyundai i10 Magna 1.2,Hyderabad,2010,74000,1.0,1.0,0.0,28.504,1197.0,78.90,5.0,3.10,hyundai i10 magna
341,344,Hyundai i10 Magna 1.2,Chennai,2009,620000,1.0,1.0,0.0,28.504,1197.0,78.90,5.0,2.70,hyundai i10 magna


In [28]:
locations = tfidf.fit_transform(df['Location'])

In [29]:
locations.shape

(5806, 11)

In [30]:
cosine_sim3 = cosine_similarity(locations)

In [31]:
def recommend_cars_with_location(location,cosine_sim=cosine_sim3):
    # Get the index of the car that matches the name
    idx = df.index[df['Location'] == location].tolist()[0]

    # Get the pairwise similarity scores with that car
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the pcars based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar cars
    sim_scores = sim_scores[1:6]

    # Get the car indices
    car_indices = [i[0] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['Name'].iloc[car_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar cars
    return recommendations_df

In [32]:
recommend_cars_with_location('Pune')

,PropertyName,SimilarityScore
6,Volkswagen Vento Diesel Comfortline,"(6, 1.0)"
12,Land Rover Freelander 2 TD4 SE,"(12, 1.0)"
36,Mercedes-Benz M-Class ML 350 CDI,"(36, 1.0)"
61,Mercedes-Benz New C-Class 200 CDI Classic,"(61, 1.0)"
66,Maruti Ertiga SHVS VDI,"(66, 1.0)"


In [33]:
df.sample()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,transformed_name
47,47,Mahindra Thar CRDe AC,Jaipur,2012,75800,0.0,1.0,2.0,23.17,2498.0,105.0,6.0,5.5,mahindra thar crde ac


In [34]:
weights = {
    'Year': 0.5,
    'Kilometers_Driven':  -0.5,
    'Owner_Type': 0.3,
    'Mileage': 0.4,
    'Engine': 0.4,
    'Power': 0.3,
    'Seats': 0.2,
}

# Calculate weighted scores
df_normalized['Score'] = (
    df_normalized['Year'] * weights['Year'] +
    df_normalized['Kilometers_Driven'] * weights['Kilometers_Driven'] +
    df_normalized['Owner_Type'] * weights['Owner_Type'] +
    df_normalized['Mileage'] * weights['Mileage'] +
    df_normalized['Engine'] * weights['Engine'] +
    df_normalized['Power'] * weights['Power'] +
    df_normalized['Seats'] * df['Seats']
)

# Sort cars by score
df_normalized = df_normalized.sort_values(by='Score', ascending=False)

# Select top 10 cars
top_10_cars = df_normalized.head(10)




In [35]:
df.iloc[top_10_cars.index]

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,transformed_name
882,887,Tata Sumo DX,Mumbai,2013,20000,0.0,1.0,0.0,19.698,1978.0,83.1,10.0,5.25,tata sumo dx
2474,2492,Chevrolet Tavera LS B3 10 Seats BSIII,Hyderabad,2015,120000,0.0,1.0,0.0,20.720,2499.0,80.0,10.0,5.50,chevrolet tavera ls b3 10 seat bsiii
1843,1852,Toyota Qualis FS B3,Bangalore,2002,63000,0.0,1.0,3.0,18.340,2446.0,75.0,10.0,3.65,toyota quali fs b3
783,788,Toyota Qualis FS B2,Pune,2004,77757,0.0,1.0,2.0,18.340,2446.0,75.0,10.0,3.50,toyota quali fs b2
2231,2244,Chevrolet Tavera LT 9 Str BS IV,Kolkata,2012,67132,0.0,1.0,0.0,19.012,2499.0,78.0,9.0,4.75,chevrolet tavera lt 9 str bs iv
2273,2289,Mahindra Xylo D2 Maxx,Chennai,2016,150000,0.0,1.0,0.0,20.930,2489.0,93.7,9.0,4.50,mahindra xylo d2 maxx
1295,1302,Mahindra Scorpio LX BS IV,Bangalore,2007,74112,0.0,1.0,0.0,16.870,2179.0,120.0,9.0,4.10,mahindra scorpio lx bs iv
1475,1484,Toyota Innova Crysta 2.4 VX MT 8S,Coimbatore,2019,11241,0.0,1.0,0.0,19.152,2393.0,147.8,8.0,22.55,toyota innova crysta vx mt 8s
2824,2847,Toyota Innova Crysta 2.8 GX AT 8S,Delhi,2017,18000,0.0,0.0,0.0,15.904,2755.0,171.5,8.0,18.25,toyota innova crysta gx 8s
4189,4220,Toyota Innova Crysta 2.4 VX MT 8S,Coimbatore,2019,31271,0.0,1.0,0.0,19.152,2393.0,147.8,8.0,21.42,toyota innova crysta vx mt 8s


In [36]:
import pickle

In [37]:
pickle.dump(cosine_sim1,open('cosine_sim1.pkl','wb'))
pickle.dump(cosine_sim2,open('cosine_sim2.pkl','wb'))
pickle.dump(cosine_sim3,open('cosine_sim3.pkl','wb'))

In [62]:
(cosine_sim2+cosine_sim1+cosine_sim3).shape

(5807, 5807)

In [71]:
def recommend_cars(model_name, top_n=5):
    cosine_sim_matrix = 0.5 * cosine_sim1 + 0.8 * cosine_sim2 + 1 * cosine_sim3
    # cosine_sim_matrix = cosine_sim3

    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[df[df['Name'] == model_name].index[0]]))

    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n + 1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n + 1]]

    # Retrieve the names of the top properties using the indices
    top_properties = df.index[top_indices].tolist()

    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'car_Name': top_properties,
        'SimilarityScore': top_scores
    })

    return recommendations_df

In [72]:
recommend_cars('Toyota Innova Crysta 2.8 GX AT 8S')

,car_Name,SimilarityScore
0,58,2.170752
1,5630,2.105711
2,105,2.043199
3,1962,1.943386
4,2808,1.937457


In [76]:
df['Power'].value_counts()

Power
74.00     235
98.60     131
73.90     125
140.00    123
88.50     112
         ... 
231.10      1
120.33      1
340.00      1
156.00      1
181.04      1
Name: count, Length: 358, dtype: int64